In [ ]:
# default_exp followers

In [ ]:
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
!pip install -qU nbdev fastcore
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"
#not deps but we need them to use nbdev and run tests
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


# Instagram followers
> Here I'll create a script to fetch new followers for an instagram account. The reason is to track followers change by country, facebook API offers only lifetime numbers which are updated daily.

https://developers.facebook.com/tools/explorer/

https://developers.facebook.com/docs/facebook-login/access-tokens/

## Facebook Auth

In [ ]:
#hide
!pip install -qU gspread python-facebook-api

In [ ]:
#hide
#test_flag_colab
%%capture

import yaml
with open(path.parent/"facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env USER_ID {data.get("USER_ID")}

In [ ]:
#export
import os
from datetime import datetime
from typing import *

import pandas as pd
from pyfacebook import IgProApi
from pyfacebook.error import *
from tomorrowswig_followers.core import *

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Get new followers with graph api

In [ ]:
#hide
!curl -i -X GET \
 "https://graph.facebook.com/v7.0/17841409206135869/insights?metric=audience_country&period=lifetime&access_token={TOKEN}"

HTTP/2 200 
vary: Accept-Encoding
etag: "87b2c7dd9c2100bbd4e89ecc658af49eb85a4deb"
x-business-use-case-usage: {"17841409206135869":[{"type":"instagram","call_count":1,"total_cputime":1,"total_time":1,"estimated_time_to_regain_access":0}]}
content-type: application/json; charset=UTF-8
facebook-api-version: v7.0
strict-transport-security: max-age=15552000; preload
pragma: no-cache
x-fb-rev: 1002449155
access-control-allow-origin: *
cache-control: private, no-cache, no-store, must-revalidate
x-fb-trace-id: GScgzS5ehUR
x-fb-request-id: A1ellyP3aWXhv8weazksj0v
expires: Sat, 01 Jan 2000 00:00:00 GMT
x-fb-debug: azxg85HSxTILSmWe/M1vF94GmlVeimf9BEfRWPUdYnqKu+z0/qYVZl7ED2kAQC5I7CsjeCsn3EGkf4zgJoRRqQ==
date: Fri, 31 Jul 2020 15:40:13 GMT
alt-svc: h3-29=":443"; ma=3600,h3-27=":443"; ma=3600

{"data":[{"name":"audience_country","period":"lifetime","values":[{"value":{"RS":56,"DE":862,"NP":37,"BD":55,"TW":124,"PT":156,"JO":41,"JP":81,"UA":39,"FR":55,"HU":98,"BR":2303,"MA":87,"SG":618,"DZ":105,"KE":

In [ ]:
api = IgProApi(app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN,
               version="5.0")

In [ ]:
try:
    response = api.get_user_insights(user_id=USER_ID, period="lifetime", metrics=["audience_country"],
                                    return_json=True)
except PyFacebookException as e:
    display(e.message)

In [ ]:
response = response[0]["values"][0]

In [ ]:
response["end_time"]

'2020-07-31T07:00:00+0000'

In [ ]:
#export
def get_followers() -> Tuple[str, Dict[str, int]]:
    api = IgProApi(
        app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN, version="5.0"
    )
    try:
        response = api.get_user_insights(
            user_id=USER_ID,
            period="lifetime",
            metrics=["audience_country"],
            return_json=True,
        )[0]["values"][0]
        return response["end_time"].split("T")[0], response["value"]
    except PyFacebookException as e:
        return (e.message, {0: 0})

In [ ]:
#export
def get_new_followers() -> Tuple[str, Dict[str, int]]:
    api = IgProApi(
        app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN, version="5.0"
    )
    try:
        response = api.get_user_insights(
                user_id=USER_ID,
                period="day",
                metrics=["follower_count"],
                return_json=True,
            )[0]["values"][0]
        return response["end_time"].split("T")[0], response["value"]
    except PyFacebookException as e:
        return (e.message, 0)

In [ ]:
get_new_followers()

('2020-07-30', 128)

## Google Sheets

### Insert new followers to a google sheet 

In [ ]:
end_time, followers = get_followers()

In [ ]:
df = get_df("History")

In [ ]:
end_time = datetime.strptime(end_time, "%Y-%m-%d").strftime("%b %d %Y")

In [ ]:
new_followers = pd.Series(followers)
new_followers.name = f"{end_time} {str(datetime.utcnow()).split('.')[0]}"
if new_followers.name not in df.columns:
    df = pd.concat([df, new_followers], axis=1)
    df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
df.sort_index(inplace=True)

In [ ]:
df

,Jul 31 2020 2020-07-31 15:40:18,Jul 31 2020 2020-07-31 15:37:18,Jul 31 2020 2020-07-31 15:29:29,Jul 31 2020 2020-07-31 13:12:14,Jul 31 2020 2020-07-31 13:11:09,Jul 31 2020 2020-07-31 07:10:58,Jul 30 2020 2020-07-31 01:11:06,Jul 30 2020 2020-07-30 19:10:36,Jul 29 2020 2020-07-29 13:10:41,Jul 29 2020 2020-07-29 07:10:37,Jul 28 2020 2020-07-29 01:10:35,Jul 28 2020 2020-07-28 19:10:33,Jul 28 2020 2020-07-28 13:10:37,Jul 28 2020 2020-07-28 07:10:44,Jul 27 2020 2020-07-28 01:10:39,Jul 27 2020 2020-07-27 19:10:35,Jul 27 2020 2020-07-27 13:10:34,Jul 27 2020 2020-07-27 07:10:41,Jul 26 2020 2020-07-27 01:10:34,Jul 26 2020 2020-07-26 19:10:34,Jul 26 2020 2020-07-26 13:10:33,Jul 26 2020 2020-07-26 07:10:33,Jul 25 2020 2020-07-26 01:10:37,Jul 25 2020 2020-07-25 19:10:35,Jul 25 2020 2020-07-25 13:10:42,Jul 25 2020 2020-07-25 07:10:37,Jul 24 2020 2020-07-25 01:10:41,Jul 24 2020 2020-07-24 19:10:41,Jul 24 2020 2020-07-24 13:10:42,Jul 24 2020 2020-07-24 07:10:34,Jul 23 2020 2020-07-24 01:10:38,Jul 23 2020 2020-07-23 19:10:30,Jul 23 2020 2020-07-23 13:11:01,Jul 23 2020 2020-07-23 07:10:57,Jul 22 2020 2020-07-23 01:10:56,Jul 22 2020 2020-07-22 19:11:01,Jul 22 2020 2020-07-22 13:11:00,Jul 22 2020 2020-07-22 07:11:00,Jul 21 2020 2020-07-22 01:11:01,Jul 20 2020 2020-07-21 01:11:01,...,Aug 28,Aug 27,Aug 26,Aug 25,Aug 20,Aug 19,Aug 18,Aug 17,Aug 16,Aug 15,Aug 14,Aug 13,Aug 12,Aug 11,Aug 10,Aug 9,Aug 8,Aug 7,Aug 6,Aug 5,Aug 4,Aug 3,Aug 2,Aug 1,Jul 31,Jul 30,Jul 29,Jul 28,Jul 27,Jul 25,Jul 24,Jul 23,Jul 22,Jul 21,Jul 18,Jul 17,Jul 16,Jul 14,Jun 23,Jun 21
AR,1062.0,1062,1062,1062,1062,1062,1062,1062,1048,1048,1048,1048,1028,1028,1028,1026,1026,1026,1026,1026,1027,1027,1027,1027,1028,1028,1028,1024,1024,1024,1024,1024,1012,1012,1012,1000,1000,1000,1000,1001,...,233,233,234,235,237,237,237,237,237,239,238,239,238,238,238,239,239,239,239,240,241,241,241,241,242,242,244,244,244,244,246,245,245,245,250,250,250,250,218,145
AU,799.0,799,799,799,799,799,799,799,794,794,794,794,787,787,787,752,752,752,752,752,738,738,738,738,739,739,739,724,724,724,724,724,724,724,724,724,724,724,724,724,...,66,68,67,68,66,66,66,66,66,66,66,64,63,63,63,63,60,53,51,50,51,51,45,42,0,42,42,0,0,0,0,0,0,0,0,0,0,0,40,41
BD,55.0,55,55,55,55,55,55,55,54,54,54,54,54,54,54,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,...,79,78,80,80,80,80,80,80,80,79,79,78,79,79,79,79,79,79,79,79,80,80,80,81,81,81,81,81,82,82,81,81,81,81,83,83,83,83,88,84
BR,2303.0,2303,2303,2303,2303,2303,2303,2303,2306,2306,2306,2306,2301,2301,2301,2272,2272,2272,2272,2272,2243,2243,2243,2243,2244,2244,2244,2235,2235,2235,2235,2235,2243,2243,2243,2247,2247,2247,2247,2250,...,1214,1200,1181,1156,1081,1071,1060,1046,1033,1011,987,971,953,951,923,904,897,888,873,863,852,831,818,810,791,775,760,742,731,684,669,654,644,627,595,566,547,504,248,164
CA,1723.0,1723,1723,1723,1723,1723,1723,1723,1729,1729,1729,1729,1727,1727,1727,1693,1693,1693,1693,1693,1682,1682,1682,1682,1683,1683,1683,1670,1670,1670,1670,1670,1673,1673,1673,1676,1676,1676,1676,1680,...,156,157,158,157,161,161,159,155,150,148,141,139,132,129,126,125,119,116,113,109,105,104,104,102,98,94,91,90,86,81,81,77,76,74,69,66,63,59,56,35
CL,522.0,522,522,522,522,522,522,522,522,522,522,522,520,520,520,521,521,521,521,521,523,523,523,523,523,523,523,522,522,522,522,522,523,523,523,524,524,524,524,526,...,212,214,215,215,217,216,215,216,216,217,217,216,216,218,219,219,219,219,219,219,219,220,220,220,220,222,222,222,222,225,226,226,226,226,226,228,227,226,229,177
CO,61.0,61,61,61,61,61,61,61,61,61,61,61,61,61,61,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,...,65,66,66,66,65,65,65,65,65,65,65,65,65,66,67,67,67,67,67,67,67,67,67,67,67,67,67,67,66,66,66,66,66,67,67,67,67,67,68,62
CR,69.0,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,...,85,85,85,85,85,85,85,85,85,84,84,85,85,85,85,85,85,85,85,85,85,85,85,85,85,86,86,86,86,87,87,88,88,88,89,89,89

In [ ]:
df = df.fillna(0).astype(int)

In [ ]:
#export
def get_updated_followers(
    df: pd.DataFrame, data: Dict[str, int], end_time: str
) -> pd.DataFrame:
    new_followers = pd.Series(data)
    end_time = datetime.strptime(end_time, "%Y-%m-%d").strftime(f"%b %d %Y{' '*16}")
    new_followers.name = f"{end_time} {str(datetime.utcnow()).split('.')[0]}"
    if new_followers.name not in df.columns:
        df = pd.concat([df, new_followers], axis=1)
        df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
    df.sort_index(inplace=True)
    return df.fillna(0).astype(int)

In [ ]:
fetched_followers = {'AR': 1001, 'AU': 724, 'BD': 55, 'BR': 2250, 'CA': 1680, 'CL': 526, 'CO': 6}
end_time = "2020-07-21"

test_followers_df = get_updated_followers(pd.DataFrame(), fetched_followers, end_time)
test_followers_df.rename(columns={test_followers_df.columns[0]: "2020-01-01" }, inplace=True)
assert {'2020-01-01': fetched_followers} == test_followers_df.to_dict()

test_followers_df = get_updated_followers(test_followers_df, fetched_followers, end_time)
test_followers_df.rename(columns={test_followers_df.columns[0]: "2020-01-02" }, inplace=True)
assert {'2020-01-02': fetched_followers,
        '2020-01-01': fetched_followers} == test_followers_df.to_dict()

In [ ]:
#export
def save_followers() -> str:
    df = get_df("History")
    end_time, followers = get_followers()
    df = get_updated_followers(df, followers, end_time)
    write_df(df, "History")
    return end_time

In [ ]:
# save_followers()

'2020-07-31'

### Get difference from the previous day

In [ ]:
df = get_df("History")

In [ ]:
df

,Jul 31 2020 2020-07-31 15:40:20,Jul 31 2020 2020-07-31 15:37:18,Jul 31 2020 2020-07-31 15:29:29,Jul 31 2020 2020-07-31 13:12:14,Jul 31 2020 2020-07-31 13:11:09,Jul 31 2020 2020-07-31 07:10:58,Jul 30 2020 2020-07-31 01:11:06,Jul 30 2020 2020-07-30 19:10:36,Jul 29 2020 2020-07-29 13:10:41,Jul 29 2020 2020-07-29 07:10:37,Jul 28 2020 2020-07-29 01:10:35,Jul 28 2020 2020-07-28 19:10:33,Jul 28 2020 2020-07-28 13:10:37,Jul 28 2020 2020-07-28 07:10:44,Jul 27 2020 2020-07-28 01:10:39,Jul 27 2020 2020-07-27 19:10:35,Jul 27 2020 2020-07-27 13:10:34,Jul 27 2020 2020-07-27 07:10:41,Jul 26 2020 2020-07-27 01:10:34,Jul 26 2020 2020-07-26 19:10:34,Jul 26 2020 2020-07-26 13:10:33,Jul 26 2020 2020-07-26 07:10:33,Jul 25 2020 2020-07-26 01:10:37,Jul 25 2020 2020-07-25 19:10:35,Jul 25 2020 2020-07-25 13:10:42,Jul 25 2020 2020-07-25 07:10:37,Jul 24 2020 2020-07-25 01:10:41,Jul 24 2020 2020-07-24 19:10:41,Jul 24 2020 2020-07-24 13:10:42,Jul 24 2020 2020-07-24 07:10:34,Jul 23 2020 2020-07-24 01:10:38,Jul 23 2020 2020-07-23 19:10:30,Jul 23 2020 2020-07-23 13:11:01,Jul 23 2020 2020-07-23 07:10:57,Jul 22 2020 2020-07-23 01:10:56,Jul 22 2020 2020-07-22 19:11:01,Jul 22 2020 2020-07-22 13:11:00,Jul 22 2020 2020-07-22 07:11:00,Jul 21 2020 2020-07-22 01:11:01,Jul 20 2020 2020-07-21 01:11:01,...,Aug 28,Aug 27,Aug 26,Aug 25,Aug 20,Aug 19,Aug 18,Aug 17,Aug 16,Aug 15,Aug 14,Aug 13,Aug 12,Aug 11,Aug 10,Aug 9,Aug 8,Aug 7,Aug 6,Aug 5,Aug 4,Aug 3,Aug 2,Aug 1,Jul 31,Jul 30,Jul 29,Jul 28,Jul 27,Jul 25,Jul 24,Jul 23,Jul 22,Jul 21,Jul 18,Jul 17,Jul 16,Jul 14,Jun 23,Jun 21
countries,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AR,1062,1062,1062,1062,1062,1062,1062,1062,1048,1048,1048,1048,1028,1028,1028,1026,1026,1026,1026,1026,1027,1027,1027,1027,1028,1028,1028,1024,1024,1024,1024,1024,1012,1012,1012,1000,1000,1000,1000,1001,...,233,233,234,235,237,237,237,237,237,239,238,239,238,238,238,239,239,239,239,240,241,241,241,241,242,242,244,244,244,244,246,245,245,245,250,250,250,250,218,145
AU,799,799,799,799,799,799,799,799,794,794,794,794,787,787,787,752,752,752,752,752,738,738,738,738,739,739,739,724,724,724,724,724,724,724,724,724,724,724,724,724,...,66,68,67,68,66,66,66,66,66,66,66,64,63,63,63,63,60,53,51,50,51,51,45,42,0,42,42,0,0,0,0,0,0,0,0,0,0,0,40,41
BD,55,55,55,55,55,55,55,55,54,54,54,54,54,54,54,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,...,79,78,80,80,80,80,80,80,80,79,79,78,79,79,79,79,79,79,79,79,80,80,80,81,81,81,81,81,82,82,81,81,81,81,83,83,83,83,88,84
BR,2303,2303,2303,2303,2303,2303,2303,2303,2306,2306,2306,2306,2301,2301,2301,2272,2272,2272,2272,2272,2243,2243,2243,2243,2244,2244,2244,2235,2235,2235,2235,2235,2243,2243,2243,2247,2247,2247,2247,2250,...,1214,1200,1181,1156,1081,1071,1060,1046,1033,1011,987,971,953,951,923,904,897,888,873,863,852,831,818,810,791,775,760,742,731,684,669,654,644,627,595,566,547,504,248,164
CA,1723,1723,1723,1723,1723,1723,1723,1723,1729,1729,1729,1729,1727,1727,1727,1693,1693,1693,1693,1693,1682,1682,1682,1682,1683,1683,1683,1670,1670,1670,1670,1670,1673,1673,1673,1676,1676,1676,1676,1680,...,156,157,158,157,161,161,159,155,150,148,141,139,132,129,126,125,119,116,113,109,105,104,104,102,98,94,91,90,86,81,81,77,76,74,69,66,63,59,56,35
CL,522,522,522,522,522,522,522,522,522,522,522,522,520,520,520,521,521,521,521,521,523,523,523,523,523,523,523,522,522,522,522,522,523,523,523,524,524,524,524,526,...,212,214,215,215,217,216,215,216,216,217,217,216,216,218,219,219,219,219,219,219,219,220,220,220,220,222,222,222,222,225,226,226,226,226,226,228,227,226,229,177
CO,61,61,61,61,61,61,61,61,61,61,61,61,61,61,61,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,60,...,65,66,66,66,65,65,65,65,65,65,65,65,65,66,67,67,67,67,67,67,67,67,67,67,67,67,67,67,66,66,66,66,66,67,67,67,67,67,68,62
CR,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,69,...,85,85,85,85,85,85,85,85,85,84,84,85

In [ ]:
change_df = df.diff(axis=1, periods=-1)

In [ ]:
change_df = change_df.fillna(0).astype(int)

In [ ]:
change_df = change_df.applymap(lambda x: f"+{x}" if x >0 else x)

In [ ]:
change_df = "(" + change_df.astype(str) + ")" + " " + df.astype(str)

In [ ]:
change_df = change_df.replace(["\(0\) 0", "\(0\) "], "", regex=True)

In [ ]:
change_history = get_worksheet("Change History")

In [ ]:
#export
def save_change():
    df = get_df("History")
    change_df = make_change(df)
    write_df(change_df, "Change History")

In [ ]:
#export
def make_change(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    change_df = change_df.applymap(lambda x: f"+{x}" if x > 0 else x)
    change_df = "(" + change_df.astype(str) + ")" + " " + df.astype(str)
    return change_df.replace(["\(0\) 0", "\(0\) "], "", regex=True)

In [ ]:
test_change_df = make_change(pd.DataFrame({"2020-07-21": [1001, 724, 55, 2250, 1680],
                                           "2020-07-20": [1004, 708, 55, 2300, 1600]},
                                          index=["AR", "AU", "BD", "BR", "CA"]))
assert test_change_df.iloc[4,:].to_list() == ["(+80) 1680", "1600"]
assert test_change_df.iloc[3,:].to_list() == ["(-50) 2250", "2300"]

In [ ]:
#export
def update(event: Dict = None, context=None,):
    get_followers()
    time = save_followers()
    save_change()
    return f"Updated followers from '{time}' in 'History' and 'Change History'"

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
